# Naive Bayes

This notebook codes a LDA Naive Bayes model using the [sex classification](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Sex_classification) example from wikipedia.

In [243]:
import numpy as np

In [350]:
labels = np.array(['male', 'male', 'male', 'male',
                   'female', 'female', 'female', 'female'])

height = np.array([6., 5.92, 5.58, 5.92, 5., 5.5, 5.42, 5.75])
weight = np.array([180, 190, 170, 165, 100, 150, 130, 150])
footsize = np.array([12, 11, 12, 10, 6, 8, 7, 9])

In [351]:
X = np.column_stack((height, weight, footsize))
y = labels

In [263]:
def prior(y, k):
    n_k = len(y[y == k])
    return n_k / len(y)

In [266]:
def mu_hat_k(X, y, k):
    mu_vect = X[y == k].mean(0)
    return mu_vect

def var_hat(X, y, k1, k2):
    n = X.shape[0]
    K = len(set(y))
    var_k1 = sum((X[y == k1] - X[y == k1].mean(0)) ** 2)
    var_k2 = sum((X[y == k2] - X[y == k2].mean(0)) ** 2)
    return 1 / n - K (var_k1 + var_k2)
    #var_vect = X[y == k].var(0)


## LDA: Single Feature 

In [267]:
def descriminate_function(X, y, k1, k2):
    var_hat = var_hat_k(X, y, k1, k2)
    mu_hat_k1 = mu_hat_k(X, y, k1)
    mu_hat_k2 = mu_hat_k(X, y, k2)
                 
    k1_descriminates = X * (mu_hat_k1 / var_hat) - (mu_hat_k1 ** 2 / (2 * var_hat)) \
                 + np.log(prior(y, k1))
    
    k2_descriminates = X * (mu_hat_k2 / var_hat) - (mu_hat_k2 ** 2 / (2 * var_hat)) \
                 + np.log(prior(y, k2))
    
    return [k1 if x else k2 for x in (k1_descriminates > k2_descriminates) ]
    

In [276]:
print(
    'height:', descriminate_function(height.reshape(8,1), y, 'male', 'female'), '\n',
    'weight:', descriminate_function(weight.reshape(8,1), y, 'male', 'female'), '\n',
    'footsize:', descriminate_function(footsize.reshape(8,1), y, 'male', 'female')
)

height: ['male', 'male', 'female', 'male', 'female', 'female', 'female', 'male'] 
 weight: ['male', 'male', 'male', 'male', 'female', 'female', 'female', 'female'] 
 footsize: ['male', 'male', 'male', 'male', 'female', 'female', 'female', 'female']


## LDA: Multi Feature

In [353]:
def descriminate_function_multi(X, y, k1, k2):
    # inverse of covariance matrix
    Sigma_inv = np.linalg.inv(np.cov(X.T))
    # mu matrix for each class
    mu_hat_k1 = mu_hat_k(X, y, k1)
    mu_hat_k2 = mu_hat_k(X, y, k2)
    
    k1_descriminates = X @ Sigma_inv @ mu_hat_k1 \
        - (0.5) * mu_hat_k1.T @ Sigma_inv @ mu_hat_k1 \
        + prior(y, k1)
    
    k2_descriminates = X @ Sigma_inv @ mu_hat_k2 \
        - (0.5) * mu_hat_k2.T @ Sigma_inv @ mu_hat_k2 \
        + prior(y, k2)
    
    return [k1 if x else k2 for x in (k1_descriminates > k2_descriminates) ]

In [349]:
descriminate_function_multi(X, y, 'male', 'female')

['male', 'male', 'male', 'male', 'female', 'female', 'female', 'female']

In [366]:
def predict(X, k1, k2, mu_vect_k1, mu_vect_k2, Sigma):
    k1_descriminates = X @ Sigma_inv @ mu_hat_k1 \
        - (0.5) * mu_hat_k1.T @ Sigma_inv @ mu_hat_k1 \
        + prior(y, k1)
    
    k2_descriminates = X @ Sigma_inv @ mu_hat_k2 \
        - (0.5) * mu_hat_k2.T @ Sigma_inv @ mu_hat_k2 \
        + prior(y, k2)
    
    return [k1 if x else k2 for x in (k1_descriminates > k2_descriminates) ]

x = np.array([[6, 130, 8], 
              [5.75, 200, 9.5]])

Sigma_inv = np.linalg.inv(np.cov(X.T))
mu_hat_k1 = mu_hat_k(X, y, 'male')
mu_hat_k2 = mu_hat_k(X, y, 'female')

predict(x, 'male', 'female', mu_hat_k1, mu_hat_k2, Sigma_inv)

['female', 'male']